In [1]:
import numpy as np
from folktables.acs import adult_filter
from folktables import ACSDataSource, BasicProblem, generate_categories
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
import scipy.optimize as opt

### 1. Load and Preprocess the data
We are going to work with the [Folktables](https://github.com/socialfoundations/folktables#quick-start-examples) dataset (*you have already worked with it*). I have chosen some variables for you, but you can add more (*if you like to*) - here is the [full list](https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2021.pdf) of variables (some of them do not exist in `ACSDataSource`). 

Today we are going to debias a regression model using the `SEX` variable. Your model should predict the *Total person's income*  (I've digitized  it in  `target_transform=lambda x: x > 25000`, you can choose another threshold).


* If you code is slow - you can subsample data (aka reduce the number of the samples)

In [2]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["CA"], download=False)

ACSIncomeNew = BasicProblem(
    features=[
        'AGEP', # include AGE
        'COW', # include class of worker
        'SCHL', # include school education
        'WKHP', # include reported working hours
        'SEX', # include sex
        # some random, possibly noisy
        'PWGTP', # person weight
        'JWMNP', # travel time to work
    ],
    target='PINCP',
    target_transform=lambda x: x > 25000,    
    group='SEX',
    preprocess=adult_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

Here is a small snippet to get the names of the categorical variables - I convert categoricals into one-hot encoded (*you don't have to, depending on what assumptions you use about the data*). **Don't forget to normalise the continious features (if you plan to use Cross-Validation features should be normalized per fold, aka not in the global table).**

In [3]:
# One-hot encode categorical columns
definition_df = data_source.get_definitions(download=False)
categories = generate_categories(features=ACSIncomeNew.features, definition_df=definition_df)
features, labels, groups = ACSIncomeNew.df_to_pandas(acs_data, categories=categories, dummies=True)

# Fill nas in data with -1s
features = features.astype(float).fillna(-1)

# Add intercept column
features.insert(0, 'ones', np.ones(len(features)))

# Test-train split
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    features.values, labels.values.reshape(-1), groups.values.reshape(-1), test_size=0.3, random_state=0, shuffle=True)

# Subsample to make code run faster
N = 1000
X_train = X_train[:N]
y_train = y_train[:N]
group_train = group_train[:N]

# Normalize continious features
scaler = RobustScaler(unit_variance=True).fit(X_train[:, 1:5]) ## normalize the first four columns, aka continiou features
X_train_scaled = np.concatenate([np.ones((X_train.shape[0],1)), scaler.transform(X_train[:,1:5]), X_train[:,5:]], axis=1)
X_test_scaled = np.concatenate([np.ones((X_test.shape[0],1)), scaler.transform(X_test[:,1:5]), X_test[:,5:]], axis=1)

### 3. Constrained Regression Model 
Now let's try to include the [Fairness Constraint](https://arxiv.org/abs/1706.02409)! You'll have to implement couple of things from scratch (as it is tricky to add a custom constraint function in `sklearn`.  To optimise the cost function let's use `scipy.optimize.fmin_tnc`. To calculate gradient you can use `fprime` attribute):
1. Logistic Regression
2. L2 penalisation
3. **Individual** Fairness Constrained

When you are finished with implementation - you should evaluate performance on multiple choices of fairness weight, $\lambda$.

#### Detailed breakdown
The INDIVIDUAL constraint constraint looks like this:

$$ 
f(\beta,S) = \left( \frac{1}{n_1 n_2} \sum_{(x_i,y_i)\in S_1, (x_j,y_j)\in S_2} d(y_i,y_j) (\beta^T  \textbf{x}_i - \beta^T \textbf{x}_j)^2  \right) 
$$


For the constrained optimization we have to solve a problem on the form:

$$ \min_\beta \left( \ell (\beta,S) + \lambda f(\beta,S)  +\gamma \Vert \beta \Vert_2 \right) $$ 

where $\ell$ is some loss function, $f$ is the constraint function, and the $\gamma \Vert \beta \Vert_2 $ is L2 regularization (we use it to avoid overfitting).
(Basically we are minimizing the Lagrangian $\mathscr{L} = \ell (\beta,S) + \lambda f(\beta,S)  +\gamma \Vert \textbf{x} \Vert_2$ with respect to $\beta$ - in ML literature $\mathscr{L}$ is often denoted as J)

Because we are doing classification we are going to use logistic regression. The log loss function is:
$$
\ell = \frac{1}{m}\sum_i^m\left[ -y_i \log(g(x_i)) - (1-y_i)\log(1-g(x_i)) \right], \text{where } g(x_i) = \frac{1}{1+\exp(-\beta_i x_i)}
$$

For the distance function we follow the approach from Berk et al. (2017) and set:
$$d(y_i,y_j) = \begin{cases}
            1, &         \text{if } y_i=y_j,\\
            0, &         \text{if } y_i\neq y_j.
    \end{cases}$$
    
To minimize the total loss function we also need to estimate the gradient of $\mathscr{L}$ with respect to $\beta$. Here to update the $\beta$ values we are just going the gradient's without the fairness constraing - this will make our lives considerably easier. The j'th element of the gradiend is defined as follows:
$$
\frac{\partial \mathscr{L}}{\partial \beta_j} \approx \frac{1}{m}\left( \sum_i  (g(x_i) - y_i) x[j] \right)+ 2\gamma \beta_j
$$

##### A little clarification and tips:
1. In order to simplify the exercise - we cut some corners. *Ideally* we should calculate the gradient in respect to the *individual fairness*. The gradient takes into the account only logistic and l2 loss (aka, parameters are updated based on those). At the same time, our *cost* has a *individual fairness* included. When the update of the parameters stops decreasing the cost, the `fmin_tnc` is going to stop optimisation. So our implementation is not entirely correct.
2. In case you want to have a more correct implementation, you can do `opt.fmin_tnc(func=compute_cost, x0=betas, fprime = None, approx_grad= True, ...)`. It is quite long, but you can still do it
3. I also suggest setting `ftol=1e-5`. 
4. Don not apply l2-regularization on the intercept (when you calculate the gradient).
5. You should include $x_0 = 1$ in your data, for each observation (when it comes to the manual implementation of logistic regression) to include bias (i.e. weight $\beta_0$).
6. To keep the exercise simpler, let's fix $\gamma = 1e-5$.
7. Try $lambda$ is a range from around $1$ to $1e5$.

In [4]:
def sigmoid(x):
    """
    Compute the sigmoid function 
    f = 1/(1+exp(-beta^T * x))
    This function assumes as input that you have already multiplied beta and X together
    """
    return 1./(1 + np.exp(-x))

def logistic_loss(y_true, y_pred, eps = 1e-9):
    """
    Loss for the logistic regression, y_pred are probabilities
    """
    return -np.mean(y_true * np.log(y_pred + eps) + (1-y_true) * np.log(1-y_pred + eps))

def l2_loss(beta):
    """
    L2-Regularisation
    """
    return np.linalg.norm(beta,2)

def fair_loss(y, y_pred, groups):
    """
    Individual fairness Loss
    """
    n = y.shape[0]
    n1 = np.sum(groups == 1)
    n2 = np.sum(groups == 2)
    #print(n, n1, n2)
    
    ## Solution with numpy
    equal_pairs = np.argwhere(y[np.newaxis, :] == y[:, np.newaxis]) ### Check if labels are the same, distance is 1 if y[i] == y[j] and 0 if y[i] != y[j]
    equal_pairs = equal_pairs[np.where(equal_pairs[:,0] != equal_pairs[:,1])] ### Remove the pairs where items are compared to themselves
    diff_groups = (groups[equal_pairs[:,0]] != groups[equal_pairs[:,1]]).astype(int) ### Find pairs where groups are different
    cost = diff_groups.dot((y_pred[equal_pairs[:,0]] - y_pred[equal_pairs[:,1]])**2)
    cost /= (n1*n2)
    return cost

    # Alternatively, you can use for loops (slower)
    cost = 0
    for i in range(n):
        for j in range(n):
            if i != j: ### Remove the pairs where items are compared to themselves
                if y[i] == y[j]: ### Check if labels are the same, distance is 1 if y[i] == y[j] and 0 if y[i] != y[j]
                    if groups[i] != groups[j]: ### Find pairs where groups are different
                        cost += (y_pred[i] - y_pred[j])**2
    cost /= (n1*n2)
    return cost

def compute_cost(beta, X, y, groups, _lambda, _gamma):
    # Compute predicted probs
    probs = sigmoid(X.dot(beta).astype(float))
    # Compute joint loss
    loss = logistic_loss(y, probs) + _lambda * fair_loss(y, X.dot(beta).astype(float), groups) + _gamma * l2_loss(beta[1:])
    return loss

def compute_gradients(beta, X, y, groups, _lambda, _gamma):
    """
    Calculate the gradient - used for finding the best beta values
    """
    # Start with empty gradient
    grad = np.zeros(beta.shape)
    
    m = len(X) # Number of training samples
    
    # Calculate the probs
    probs = sigmoid(X.dot(beta).astype(float))
    
    # Calculate gradients for each beta value
    for j in range(len(grad)):
        if j == 0: ### We do not want to regularize the intercept
            grad[j] = (1/m) * (probs-y).dot(X[:,j])
        else:
            grad[j] = (1/m) * (probs-y).dot(X[:,j]) + (2*_gamma)*beta[j]
        
    return grad

In [5]:
# Compute cost with random beta-values and parameters
np.random.seed(42)
compute_cost(
    beta = np.random.rand(X_train_scaled.shape[1]),
    X = X_train_scaled, 
    y = y_train,
    groups = group_train, 
    _gamma = 1, 
    _lambda = 0.1
)

np.float64(4.901305222341556)

In [6]:
# Set seed and define params
np.random.seed(23)
beta = np.random.rand(X_train_scaled.shape[1])
lambda_ = 0.001
gamma_ = 1e-5 

# Run optimization
result, _, _ = opt.fmin_tnc(
    func=compute_cost,
    x0=beta,
    fprime=compute_gradients,
    approx_grad=False,
    maxfun = 500,
    args = (
        X_train_scaled, 
        y_train,
        group_train,
        lambda_, 
        gamma_
    ),
    xtol=1e-7,
    ftol=1e-5
)

  NIT   NF   F                       GTG
    0    1  5.816787271793167E-01   5.41866022E-02
    1    5  4.846767052915962E-01   3.43108908E-03
    2   16  4.302247506864266E-01   2.07848908E-03
    3   19  4.283401670230523E-01   5.57461411E-05
tnc: fscale = 91.4987
    4   25  4.261506320067870E-01   2.47960901E-05
    5   29  4.239126096694614E-01   6.02051714E-05
    6   36  4.227295912366610E-01   8.64761091E-07
    7   39  4.225778677821582E-01   6.16159871E-06
    8   44  4.222233460959509E-01   2.32055110E-07
tnc: |fn-fn-1] = 0 -> convergence
    9   85  4.222233460959509E-01   2.32055110E-07
tnc: Converged (|f_n-f_(n-1)| ~= 0)


In [7]:
from sklearn.metrics import accuracy_score

y_pred_fair = np.array([True if x >=0.5 else False for x in sigmoid(result @ X_test_scaled.T)])

accuracy_score(y_test, y_pred_fair)

0.7947529812606473

In [8]:
#Basic model to compare
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(X_train_scaled, y_train)
y_pred_clf = clf.predict(X_test_scaled)
accuracy_score(y_test, y_pred_clf)

0.7969505962521295

In [9]:
print('Fair')
print(f'Group 1: {accuracy_score(y_test[group_test==1], y_pred_fair[group_test==1]):.3f}\nGroup 2: {accuracy_score(y_test[group_test==2], y_pred_fair[group_test==2]):.3f}\n')
print('Basic')
print(f'Group 1: {accuracy_score(y_test[group_test==1], y_pred_clf[group_test==1]):.3f}\nGroup 2: {accuracy_score(y_test[group_test==2], y_pred_clf[group_test==2]):.3f}')

Fair
Group 1: 0.801
Group 2: 0.788

Basic
Group 1: 0.805
Group 2: 0.788
